BA = Basveeling Andrew


https://github.com/mjpyeon/wavenet-classifier/blob/master/WaveNetClassifier.py

Dataset is the raw RADAR data that has not been reshaped into chirps.

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [0]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/wavenet/range_fft_bins_5_25_mti/'
MODEL_PATH = BASE_PATH + 'models/wavenet/range_fft/bins_5_25_mti/test_4/'
RESULTS_PATH = BASE_PATH + 'results/wavenet/range_fft/'
    
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory
# from src.models.dilated_CNN.wavenet_basveeling_andrew import WaveNetClassifier
from src.data_generator.data_generator import DataGenerator

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix
from keras import metrics
from keras import optimizers
from keras.callbacks import History, ModelCheckpoint, CSVLogger
from keras.models import load_model

In [0]:
# Load in data dictionary.
# This does not load in any actual data,
# just the dictionary with the names of the files and their associated labels
with open(DATA_PATH + "index.pkl", "rb") as file:
    data = pickle.load(file)

In [0]:
# Remove user C as this user is reserved for the test set
# try:
#     del data["C"]
# except KeyError:
#     print ("Key 'C' not found")

In [0]:
def convert_label_to_int(label):
    if label == "walking":
        return 0
    if label == "pushing":
        return 1
    if label == "sitting":
        return 2
    if label == "pulling":
        return 3
    if label == "circling":
        return 4
    if label == "clapping":
        return 5
    if label == "bending":
        return 6

In [0]:
labels = {}
partition = {'train':[], 'validation':[]} # contains list of training and validation ID's
validation_user = "C"

for user_letter, actions in data.items():
    for action, results in actions.items():
        for result in results:
            for row in result:
                if user_letter == validation_user:
                    partition["validation"].append(row)
                    labels[row] = convert_label_to_int(action)

                else:
                    partition["train"].append(row)
                    labels[row] = convert_label_to_int(action)

In [0]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
nb_classes = len(target_names)

In [0]:
data_shape = (3000, 19)
n_filters = 64
dilation_depth = 8
activation = 'softmax'
scale_ratio = 1
kernel_size = 2
pool_size_1 = 4
pool_size_2 = 8
batch_size = 16
epochs = 20

In [0]:
# Parameters
params = {'dim': (data_shape),
          'batch_size': batch_size,
          'n_classes': nb_classes,
          'data_directory': DATA_PATH}
# Generators
training_generator = DataGenerator(partition['train'], labels, **params, shuffle=True)
validation_generator = DataGenerator(partition['validation'], labels, **params, shuffle=False, data_type="Validation")

In [0]:
'''
Wavenet model from https://github.com/basveeling/wavenet adapted to
handle classification based on the methods described in the original wavenet paper
'''

from keras.layers import Input, Conv1D, Multiply, Add, Reshape, Activation, AveragePooling1D, Lambda, Flatten, Dense
from keras.models import load_model, Model
from keras.callbacks import History, ModelCheckpoint

import pandas as pd
import sys
import tensorflow as tf


class WaveNetClassifier:
    def __init__(self, input_shape, output_shape, kernel_size=2, dilation_depth=9, nb_stacks=1, nb_filters=40,
                 pool_size_1=80, pool_size_2=100, use_bias=False, use_skip_connections=False):
        """
        Parameters:
          input_shape: (tuple) tuple of input shape. (e.g. If input is 6s raw waveform with sampling rate = 16kHz, (96000,) is the input_shape)
          output_shape: (tuple)tuple of output shape. (e.g. If we want classify the signal into 100 classes, (100,) is the output_shape)
          kernel_size: (integer) kernel size of convolution operations in residual blocks
          dilation_depth: (integer) type total depth of residual blocks
          n_filters: (integer) # of filters of convolution operations in residual blocks
          load: (bool) load previous WaveNetClassifier or not
          load_dir: (string) the directory where the previous model exists
        """
        self.activation = 'softmax'
        self.scale_ratio = 1
        self.pool_size_1 = pool_size_1
        self.pool_size_2 = pool_size_2
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.dilation_depth = dilation_depth
        self.nb_filters = nb_filters
        self.use_bias = use_bias
        self.use_skip_connections = use_skip_connections
        self.input_shape = input_shape
        self.output_shape = output_shape

        if len(input_shape) == 1:
            self.expand_dims = True
        elif len(input_shape) == 2:
            self.expand_dims = False
        else:
            print('ERROR: wrong input shape')
            sys.exit()

        self.model = self.build_model()

    def residual_block(self, x, i, stack_nb):
        original_x = x
        tanh_out = Conv1D(self.nb_filters, 2, dilation_rate=2 ** i, padding='causal',
                          use_bias=self.use_bias,
                          name='dilated_conv_%d_tanh_s%d' % (2 ** i, stack_nb), activation='tanh')(x)
        sigm_out = Conv1D(self.nb_filters, 2, dilation_rate=2 ** i, padding='causal',
                          use_bias=self.use_bias,
                          name='dilated_conv_%d_sigm_s%d' % (2 ** i, stack_nb), activation='sigmoid')(x)
        x = Multiply(name='gated_activation_%d_s%d' % (i, stack_nb))([tanh_out, sigm_out])

        res_x = Conv1D(self.nb_filters, 1, padding='same', use_bias=self.use_bias)(x)
        skip_x = Conv1D(self.nb_filters, 1, padding='same', use_bias=self.use_bias)(x)
        res_x = Add()([original_x, res_x])
        return res_x, skip_x

    def build_model(self):
        input_layer = Input(shape=self.input_shape, name='input_part')
        out = input_layer
        skip_connections = []
        out = Conv1D(self.nb_filters, 2,
                     dilation_rate=1,
                     padding='causal',
                     name='initial_causal_conv'
                     )(out)
        for stack_nb in range(self.nb_stacks):
            for i in range(0, self.dilation_depth + 1):
                out, skip_out = self.residual_block(out, i, stack_nb)
                skip_connections.append(skip_out)

        if self.use_skip_connections:
            out = Add()(skip_connections)
        out = Activation('relu')(out)
        # added a mean-pooling layer after the dilated convolutions that aggregated the activations to coarser frames
        # spanning 10 milliseconds (160× downsampling)
        # mean pooling layer adjust pool_size_1 to change downsampling
        out = AveragePooling1D(self.pool_size_1, padding='same', name='mean_pooling_layer_downsampling')(out) # 4x downsampling


        # few non-causal convolutions
        out = Conv1D(self.nb_filters, self.pool_size_1, strides=2, padding='same', activation='relu')(out)
        out = Conv1D(self.nb_filters, self.pool_size_2, strides=2, padding='same', activation='relu')(out)
        out = Conv1D(self.output_shape, self.pool_size_2, strides=2, padding='same', activation='relu')(out)
        out = Conv1D(self.output_shape, self.pool_size_2, strides=2, padding='same', activation='relu')(out)


        out = Flatten()(out)
        out = Dense(512, activation='relu')(out)
        # add dropout?
        out = Dense(self.output_shape, activation='softmax')(out)
        
        
#                 out = Conv1D(self.n_filters, self.pool_size_1, strides=1, padding='same',
#                      activation='relu', name='conv_5ms')(out)
#         out = AveragePooling1D(self.pool_size_1, padding='same', name='downsample_to_200Hz')(out)

#         out = Conv1D(self.n_filters, self.pool_size_2, padding='same',
#                      activation='relu', name='conv_500ms')(out)
#         out = Conv1D(self.output_shape[0], self.pool_size_2, padding='same',
#                      activation='relu', name='conv_500ms_target_shape')(
#             out)
#         out = AveragePooling1D(self.pool_size_2, padding='same', name='downsample_to_2Hz')(out)
#         out = Conv1D(self.output_shape[0], (int)(self.input_shape[0] / (self.pool_size_1 * self.pool_size_2)), padding='same',
#                      name='final_conv')(out)
#         out = GlobalAveragePooling1D(name='final_pooling')(out)
#         out = Activation(self.activation, name='final_activation')(out)


        return Model(input_layer, out)

    def get_model(self):
        return self.model

    def get_summary(self):
        self.model.summary()

    def get_receptive_field(self):
        return self.nb_stacks * (2 ** self.dilation_depth * 2) - (self.nb_stacks - 1)


In [0]:
wnc = WaveNetClassifier(data_shape, (7), kernel_size=2, dilation_depth=4, nb_stacks=1, nb_filters=64, pool_size_1=4, pool_size_2=8, use_bias=False, use_skip_connections=True)

In [0]:
wnc.get_summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_part (InputLayer)         (None, 3000, 19)     0                                            
__________________________________________________________________________________________________
initial_causal_conv (Conv1D)    (None, 3000, 64)     2496        input_part[0][0]                 
__________________________________________________________________________________________________
dilated_conv_1_tanh_s0 (Conv1D) (None, 3000, 64)     8192        initial_causal_conv[0][0]        
__________________________________________________________________________________________________
dilated_conv_1_sigm_s0 (Conv1D) (None, 3000, 64)     8192        initial_causal_conv[0][0]        
__________________________________________________________________________________________________
gated_acti

In [0]:
wnc = WaveNetClassifier(data_shape, (7,), kernel_size = kernel_size,
                        dilation_depth = dilation_depth, n_filters = n_filters,
                        pool_size_1=pool_size_1, pool_size_2=pool_size_2)

In [0]:
model = wnc.get_model()

In [0]:
load_weights = False
weights_path = MODEL_PATH + "epoch-05-val_acc-0.77.hdf5"

start_epoch = 0
if load_weights:
    model = load_model(weights_path)
#     model.load_weights(weights_path)
    last_epoch = weights_path.split("-")[-3]
    start_epoch = int(last_epoch)

In [0]:
if not load_weights:
    model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
checkpoint = ModelCheckpoint(MODEL_PATH + "epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5",
                             monitor='val_acc', verbose=0, save_best_only=False,
                             save_weights_only=False, mode='auto', period=1)

csv_logger = CSVLogger(RESULTS_PATH + "test_4.csv", append=True)
callbacks_list = [checkpoint, csv_logger]

In [0]:
# Train model on dataset
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=9,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    initial_epoch=start_epoch)

Epoch 1/20
2315/2315 [==============================] - 522s 226ms/step - loss: 1.3438 - acc: 0.4094 - val_loss: 1.0707 - val_acc: 0.5570

Epoch 2/20
2315/2315 [==============================] - 506s 219ms/step - loss: 0.6498 - acc: 0.7089 - val_loss: 0.9194 - val_acc: 0.6530
Epoch 3/20
2315/2315 [==============================] - 508s 219ms/step - loss: 0.3257 - acc: 0.8702 - val_loss: 1.3848 - val_acc: 0.5692
Epoch 4/20
2315/2315 [==============================] - 506s 218ms/step - loss: 0.2530 - acc: 0.9168 - val_loss: 0.9152 - val_acc: 0.6880
Epoch 5/20
2315/2315 [==============================] - 507s 219ms/step - loss: 0.1677 - acc: 0.9381 - val_loss: 0.7156 - val_acc: 0.7728
Epoch 6/20
 268/2315 [==>...........................] - ETA: 6:20 - loss: 0.1370 - acc: 0.9499

Process ForkPoolWorker-97:
Process ForkPoolWorker-107:
Process ForkPoolWorker-94:
Process ForkPoolWorker-106:
Process ForkPoolWorker-108:
Process ForkPoolWorker-96:
Process ForkPoolWorker-95:
Process ForkPoolWorker-93:
Process ForkPoolWorker-92:
Process ForkPoolWorker-98:
Process ForkPoolWorker-91:
Process ForkPoolWorker-100:
Process ForkPoolWorker-105:
Traceback (most recent call last):
Process ForkPoolWorker-104:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-102:
Process ForkPoolWorker-101:
Process ForkPoolWorker-103:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-99:
Traceback (most recent call last):
  File "/u

KeyboardInterrupt: ignored

In [0]:
visualize.plot_train_test_acc(history=history)

In [0]:
visualize.plot_train_test_loss(history=history)